In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("/kaggle/input/churn-for-bank-customers/churn.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df=df.drop(['RowNumber', 'CustomerId','Surname'], axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.duplicated().sum()

In [ ]:
df.rename(columns={'Exited':'Churn'}, inplace=True)

In [ ]:
df.describe()

In [ ]:
df.head()

# Explorative Data Analysis

In [ ]:
plt.figure(figsize=(10,6))
plt.pie(df['Churn'].value_counts(),labels=['No','Yes'],autopct='%1.1f%%')
plt.title('Churn Percentage')
plt.show()

In [ ]:
sns.countplot(x='Gender',data=df,hue='Churn')
plt.title('Gender Distribution')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()



In [ ]:
sns.histplot(data=df,x="Age",hue="Churn", multiple='stack',kde=True)

In [ ]:
fig, ax =plt.subplots(1,2,figsize=(15,5))
sns.boxplot(x="Churn", y="CreditScore", data=df,ax=ax[0])
sns.violinplot(x="Churn",y="CreditScore",data=df,ax=ax[1])

In [ ]:
sns.countplot(x="Geography",hue="Churn",data=df)
plt.title('Geography and Churn')
plt.xlabel('Geography')
plt.ylabel('Churn')
plt.show()

In [ ]:
fig, ax=plt.subplots(1,2, figsize=(15,5))
sns.countplot(x='Tenure',data=df, ax=ax[0])
sns.countplot(x='Tenure', hue='Churn',data=df, ax=ax[1])


In [ ]:
sns.histplot(data=df,x="Balance",hue="Churn", multiple="stack", kde=True)

In [ ]:
sns.countplot(x='NumOfProducts',hue="Churn",data=df)

In [ ]:
sns.countplot(x=df['HasCrCard'], hue=df['Churn'])

In [ ]:
sns.countplot(x='IsActiveMember', hue='Churn', data=df)

In [ ]:
sns.histplot(data=df,x='EstimatedSalaty',hue,'Churn',multiple='stack',palette=None)

In [ ]:
#Label encoding the variables
variables=['Geography','Gender']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for i in variables:
    le.fit(df[i].unique())
    df[i]=le.transform(df[i])
    print(i,df[i].unique())
    

In [ ]:
#normalization
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df[['CreditScore','Balance','EstimatedSalary']]=scaler.fit_transform(df[['CreditScore','Balance','EstimatedSalary']])

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_Test=train_test_split(df.drop('Churn',axis=1),df['Churn'],test_size=0.3,random_state=42)

In [ ]:
#churn prediction
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dtree=DecisionTreeClassifier()

#definin parameter range 
param_grid={
    'max_depth':[2,4,6,8,10,12,14,16,19,20],
    'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10],
    'criterion':['gini','entropy'],
    'random_state':[0,42]
}

#creating grid search object 
grid_dtree=GridSearchCV(dtree,param_grid, cv=5, scoring='roc_auc', n_jobs=-1,
                       verbose=1)
#fitting the grid search object to the training data 
grid_dtree.fit(X_train,y_train)

print('BEST PARAMETERS FOUND:',grid_dtree.best_params_)

In [ ]:
dtree=DecisionTreeClassifier(criterion='gini',max_depth=6,random_state=42,min_samples_leaf=10)
dtree

In [ ]:
#training the model

dtree.fit(X_train,y_train)

#training the accuracy
dtree.score(X_train,y_train)

In [ ]:
dtree_pred=dtree.predict(X_test)
dtree_pred

In [ ]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
#creating Random Forest Classifier Object

rfc=RandomForestClassifier()

#defining parameter Range
param_grid={
    'max_depth':[2,4,6,8,10],
    'min_samples_leaf':[2,4,6,8,10],
    'criterion' :['gini', 'entropy'],
    'random_state':[0,42]
}

#Creating Grid Search Object 
grid_rfc=GridSearchCV(rfc,param_grid,cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)

#fitting the grid search object to the training data

grid_rfc.fit(X_train, y_train)
#printing the best params
print('Best Parameters Found:', grid_rfc.best_params_)

In [ ]:
#Adding the parameters to the model
rfc=RandomForestClassifier(min_samples_leaf=8, max_depth=10, random_state=0,criterion='entropy')
rfc

training the model
rfc.fit(X_train,y_train)
#model accuracy
rfc.score(X_test,y_Test)

In [ ]:
rfc_pred=rfc.predict(X_test)

In [ ]:
#Model Evulation
#Decision Tree Classifier
#Confusion Matrix Heatmap

from sklearn.metrics import confusion_matrix
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_Test,dtree_pred), annot=True, fmt='d',cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Conf. Matrix for Decision Tree')
plt.show()

In [ ]:
### distribution plot
ax=sns.distplot(y_Test,hist=False,color='r',label='Actual Val.')
sns.distplot(dtree_pred,hist=False, color='b',label='Fitted Val',ax=ax)

In [ ]:
#Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_Test,dtree_pred))

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score
print("Accuracy Score: ", accuracy_score(y_Test, dtree_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_Test, dtree_pred))
print("R2 Score: ", r2_score(y_Test, dtree_pred))

# Random Forest Classifier
# Confusion Matrix Heatmap

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_Test,rfc_pred),annot=True,fmt='d',cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for Random Forest Classifier')
plt.show()

In [ ]:
#Distribution Plot
ax = sns.distplot(y_Test, hist=False, color="r", label="Actual Value")
sns.distplot(rfc_pred, hist=False, color="b", label="Fitted Values" , ax=ax)

In [ ]:
#Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_Test, rfc_pred))

In [ ]:
print("Accuracy Score: ", accuracy_score(y_Test, rfc_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_Test, rfc_pred))
print("R2 Score: ", r2_score(y_Test, rfc_pred))

Conclusion
From the exploratory data analysis, I have concluded that the churn count of the customersdepends upon the following factors:

Age
Geography
Tenure
Balance
Number of Products
Has Credit Card
Is Active Member
Coming to the classification models, I have used the following models:

Decision Tree Classifier
Random Forest Classifier
Both the models were hyperparameter tuned using GridSearchCV. Both the models have nearly equal accuracy score. But, the Random Forest Classifier has a better accuracy and precision score than the Decision Tree Classifier.